In [51]:
import requests
from bs4 import BeautifulSoup
import sqlite3

def scrape_job_details(url):
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')

        # 仕事タイトルの抽出
        job_title_tag = soup.find("title")
        job_title = job_title_tag.text.strip().split("のアルバイト")[0] if job_title_tag else "N/A"

        # 職種の抽出
        job_description = "N/A"
        job_description_section = soup.find("div", class_="detail-basicInfo")
        if job_description_section:
            description_tag = job_description_section.find("dd")
            if description_tag:
                job_description = " ".join([p.get_text(strip=True) for p in description_tag.find_all("p")])
        
        # 時給の抽出
        hourly_wage = "N/A"
        hourly_wage_section = soup.find_all("em")
        for wage_tag in hourly_wage_section:
            if "時給" in wage_tag.get_text(strip=True):
                hourly_wage = wage_tag.get_text(strip=True).replace("時給", "").strip()
                break

        # 最寄駅の抽出
        nearest_station = "N/A"
        nearest_station_section = soup.find("dt", string="最寄駅")
        if nearest_station_section:
            nearest_station_dd = nearest_station_section.find_next("dd")
            if nearest_station_dd:
                stations = [line.strip() for line in nearest_station_dd.decode_contents().split("<br/>")]
                nearest_station = "・".join(stations).replace("\n", "").strip()

        # 住所の抽出
        address = "N/A"
        address_section = soup.find("dt", string="住所")
        if address_section:
            address_dd = address_section.find_next("dd")
            if address_dd:
                address = address_dd.get_text(strip=True).replace("\n", ", ")

        # 結果の準備
        result = {
            "仕事タイトル": job_title,
            "職種": job_description,
            "時給": hourly_wage,
            "最寄駅": nearest_station,
            "住所": address,
        }

        # 結果の表示
        print("スクレイピング結果:")
        print(f"仕事タイトル: {result['仕事タイトル']}")
        print(f"職種: {result['職種']}")
        print(f"時給: {result['時給']}")
        print(f"最寄駅: {result['最寄駅']}")
        print(f"住所: {result['住所']}")
        print("-" * 50)

        return result
    else:
        print(f"ウェブページの取得に失敗しました。ステータスコード: {response.status_code}")
        return None


def save_to_database(data, db_name="job_data.db"):
    conn = sqlite3.connect(db_name)
    cursor = conn.cursor()
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS jobs (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            job_title TEXT,
            job_description TEXT,
            hourly_wage TEXT,
            nearest_station TEXT,
            address TEXT
        )
    ''')
    cursor.execute('''
        INSERT INTO jobs (job_title, job_description, hourly_wage, nearest_station, address)
        VALUES (?, ?, ?, ?, ?)
    ''', (data["仕事タイトル"], data["職種"], data["時給"], data["最寄駅"], data["住所"]))
    conn.commit()
    conn.close()
    print("ジョブがデータベースに保存されました:", db_name)

def display_all_jobs(db_name="job_data.db"):
    conn = sqlite3.connect(db_name)
    cursor = conn.cursor()
    cursor.execute('SELECT * FROM jobs')
    rows = cursor.fetchall()
    print("現在のデータベースの内容:")
    for row in rows:
        print(f"ID: {row[0]}, 仕事タイトル: {row[1]}, 仕事内容: {row[2]}, 時給: {row[3]}, 最寄駅: {row[4]}, 住所: {row[5]}")
    conn.close()

url = "https://www.baitoru.com/kanto/jlist/tokyo/23ku/shinjukuku/job136194735/"
result = scrape_job_details(url)

if result:
    save_to_database(result)
    display_all_jobs()


スクレイピング結果:
仕事タイトル: 【新店】おかげ庵新宿センタービル店
職種: コメダ珈琲店　ホール＆キッチン [ア・パ]ホールスタッフ(配膳)、キッチンスタッフ、案内(インフォメーション/レセプション)・フロント
時給: 1,300円～1,625円
最寄駅: 山手線　新宿駅　徒歩4分・小田急小田原線　新宿駅　徒歩6分・東京メトロ丸ノ内線　西新宿駅　徒歩8分
住所: 東京都新宿区西新宿1‐25‐1 新宿センタービル地下1階新宿駅西口、19番出口から徒歩4分、新宿センタービル地下1階にございます
--------------------------------------------------
ジョブがデータベースに保存されました: job_data.db
現在のデータベースの内容:
ID: 1, 仕事タイトル: 株式会社ヤマダデンキ　LABI新宿西口館W1100/P0037, 仕事内容: 家電量販店 [ア・パ]レジ打ち, 時給: 1,250円～1,450円, 最寄駅: 中央本線　新宿駅　徒歩5分・山手線　新宿駅　徒歩5分・埼京線　新宿駅　徒歩5分, 住所: 東京都新宿区西新宿一丁目18-8
ID: 2, 仕事タイトル: 株式会社ディンプルの派遣の求人情報｜バイトルで仕事探し(No.134211425), 仕事内容: 百貨店(デパート) [派遣]イベントスタッフ、デリ・惣菜販売・スイーツ販売、レジ打ち, 時給: 1,600円～, 最寄駅: 山手線　新宿駅　徒歩1分・埼京線　池袋駅　徒歩1分・横浜線　町田駅　徒歩1分, 住所: 東京都新宿区首都圏の百貨店・駅ビルで大募集！勤務地の相談ももちろんOK！＜東京＞大丸東京GINZA SIX松屋銀座松坂屋上野新宿伊勢丹新宿京王池袋東武小田急町田玉川高島屋＜神奈川＞横浜高島屋赤レンガ倉庫＜埼玉＞大宮そごう浦和伊勢丹川越マルヒロ　　etc...学校の近く、家の近く、職場の近くなどご希望をお聞かせください◎
ID: 3, 仕事タイトル: ゲオモバイル LICOPA東大和店, 仕事内容: スマホ・携帯販売店 [ア・パ]携帯販売、レジ打ち、サービスその他, 時給: 1,200円～, 最寄駅: 多摩都市モノレール　桜街道駅　徒歩10分, 住所: 東京都東大和市桜が丘2丁目142番1号 LICOPA東大和2F
ID: 4, 仕事タイトル: